# Ingeniería de Datos – ETLs y Workflows con Delta Lake
## Guía práctica (clase)

Este notebook es la **guía teórico-práctica** del bloque.
Aquí **NO se programa en profundidad**, se explican conceptos y ejemplos.

---

## 🎯 Objetivo del bloque

Entender cómo pasar:
- de **datos crudos**
- a **pipelines ETL robustos**
- listos para **producción** con Delta Lake y Workflows


%md
## 1️⃣ Recordatorio ETL

**ETL = Extract – Transform – Load**

Un ETL (Extract, Transform, Load) es un proceso fundamental en la ingeniería de datos. Consiste en extraer datos de diferentes fuentes, transformarlos para adecuarlos a las necesidades del negocio y cargarlos en un sistema de almacenamiento o análisis, como un Data Lake o un Data Warehouse.

### Extract
En los entornos de Big Data, los datos pueden venir en múltiples formatos. Cada formato tiene ventajas y desventajas en cuanto a compresión, velocidad de lectura/escritura y compatibilidad. Parquet, por ejemplo, es columnar y eficiente para grandes volúmenes.
- CSV
- Parquet
- JSON
- Bases de datos
- Streaming

### Transform
- Limpieza
- Reglas de negocio
- Normalización
- Agregaciones

### Load
- Data Lake
- Data Warehouse
- Tablas Delta


#### Leer desde CSV
Leer datos desde un archivo CSV es una de las formas más comunes de ingesta en proyectos de datos. El formato CSV (Comma Separated Values) es ampliamente utilizado por su simplicidad y compatibilidad con la mayoría de las herramientas. Pero no es el formato más eficiente para grandes volúmenes de datos.

<pre>
df_csv = spark.read.csv(
    path=path,
    header=...,
    inferSchema=...,
    sep=...
)
</pre>

#### Leer desde parquet
El formato Parquet es un estándar de almacenamiento columnar ampliamente utilizado en Big Data. Su principal fortaleza es la eficiencia tanto en almacenamiento como en velocidad de lectura, especialmente cuando se trabaja con grandes volúmenes de datos y consultas sobre columnas específicas. Parquet permite compresión y soporta tipos de datos complejos, lo que lo hace ideal para análisis y procesamiento distribuido. 

<pre>
df_parquet = spark.read.parquet(path)
</pre>

#### Leer desde Json
El formato JSON (JavaScript Object Notation) es ampliamente utilizado para el intercambio de datos debido a su flexibilidad y legibilidad. Permite almacenar estructuras de datos complejas, como listas y diccionarios anidados. Pero no es tan eficiente en almacenamiento ni en velocidad de procesamiento para grandes volúmenes de datos

<pre>
df_json = spark.read.json(path)
</pre>

%md
 Fuente   | ✔️ Ventajas                                                                 | ❌ Desventajas                                                                | Uso Común                                               |
----------|-------------------------------------------------------------------------|----------------------------------------------------------------------------|---------------------------------------------------------|
 CSV      | Simple y ampliamente compatible. Fácil de editar manualmente.            | No soporta tipos de datos complejos ni compresión nativa. Menos eficiente para grandes volúmenes. | Ingesta inicial de datos pequeños o de fuentes externas.|
 Parquet  | Almacenamiento columnar eficiente, compresión nativa, rápido para consultas en columnas específicas. | No es legible por humanos. Requiere herramientas específicas para edición.  | Procesamiento de Big Data, análisis y almacenamiento optimizado.|
 JSON     | Flexible para estructuras complejas (anidadas). Legible y compatible con APIs web. | Menos eficiente en almacenamiento y procesamiento para grandes volúmenes.   | Intercambio de datos con sistemas web o cuando se necesita flexibilidad en la estructura.|
 BD       | Acceso directo a datos actualizados, integración con sistemas empresariales, soporte para consultas SQL. | Requiere configuración de conexión, puede tener limitaciones de rendimiento y permisos. | Análisis de datos operacionales, integración de datos de negocio.|

### Recordatorio: Leer con/sin esquema

Al leer datos en Spark, se puede dejar que el sistema infiera automáticamente el esquema (tipos de datos de cada columna) o definirlo explícitamente. Inferir el esquema es cómodo y rápido para exploraciones iniciales, pero puede ser más lento y propenso a errores si los datos son inconsistentes o si hay muchas columnas. Definir el esquema manualmente garantiza que los tipos de datos sean los esperados, mejora el rendimiento en la carga y ayuda a evitar problemas en etapas posteriores del procesamiento. Es una buena práctica definir el esquema en entornos productivos o cuando se requiere mayor control y robustez sobre los datos.

Ejecuta el siguiente código para leer sin esquema

%md
| Método de Lectura         | ✔️ Ventajas                                                                 | ❌ Desventajas                                                            | Uso Recomendado                          |
|--------------------------|--------------------------------------------------------------------------|------------------------------------------------------------------------|------------------------------------------|
| **Sin esquema (inferSchema=True)** | - Rápido para exploración inicial<br>- No requiere conocer los tipos de datos previamente | - Puede inferir tipos incorrectos si los datos son inconsistentes<br>- Más lento en archivos grandes<br>- Menos robusto en producción | Exploración, pruebas rápidas, datos pequeños o desconocidos |
| **Con esquema definido** | - Tipos de datos consistentes y controlados<br>- Mejor rendimiento en la carga<br>- Evita errores por inferencia incorrecta | - Requiere conocer la estructura de los datos<br>- Más trabajo inicial | Procesos productivos, datos críticos, ETLs, grandes volúmenes |

<pre>
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Definir el esquema manualmente
schema = StructType([
    # Nombre, Tipo de dato, Requerido
    StructField("id", StringType(), True),
    StructField("fecha", StringType(), True),
    StructField("producto", StringType(), True),
    StructField("cantidad", IntegerType(), True),
    StructField("precio", DoubleType(), True)
])

df_csv = spark.read.csv(
    path=base_path+"ventas.csv",
    header=True,
    schema=schema,
    sep=","
)

df_csv.printSchema()
</pre>

%md
## 2️⃣ Tipos de extracción

### Carga total
- Se borra y se vuelve a cargar todo
- Simple
- Poco eficiente

### Carga incremental
- Solo datos nuevos
- Basada en fecha o ID
- Más compleja, más eficiente

👉 **En producción casi siempre incremental**

Copia el siguiente bloque de código en el notebook adaptando las rutas


<pre>
# Ejemplo de carga total
df_total = spark.read.csv("/Volumes/master/repaso/datos/total.csv", header=True, inferSchema=True)
display(df_total)

# Ejemplo de carga incremental
df_incremental = spark.read.csv("/Volumes/master/repaso/datos/incremental.csv", header=True, inferSchema=True)
df_incremental_load = df_incremental.join(df_total, on=df_total.columns, how="left_anti")
display(df_incremental_load)
</pre>

%md
## 3️⃣ Transformaciones habituales

### Limpieza
- Eliminar nulos
- Eliminar duplicados

### Estandarización
- Tipos de datos
- Formatos de fecha

### Reglas de negocio
- Columnas calculadas
- Flags


Ve copiando cada celda en el notebook y comprueba como va cambiando el df

<pre>
# Ejemplo de transformaciones habituales en Spark DataFrames

from pyspark.sql.functions import col, trim, lower, when, regexp_replace

# Creamos un DataFrame de ejemplo
data = [
    (1, "  Juan  ", "M", "Madrid", 25, None),
    (2, "Ana", "F", "Barcelona", 30, "2026-01-07"),
    (3, "Pedro", None, "Valencia", None, "2025-12-31"),
    (4, "lucía", "F", "Madrid", 22, "2026-01-01"),
    (5, "Carlos", "M", "Sevilla", 40, "2026-01-05"),
    (6, "  MARÍA", "F", "Madrid", 35, None)
]
columns = ["id", "nombre", "genero", "ciudad", "edad", "fecha_registro"]
df = spark.createDataFrame(data, columns)
</pre>

<pre>
# 1. Limpieza: eliminar espacios y valores nulos
df_limpio = df.withColumn("nombre", trim(col("nombre"))) \
              .na.fill({"genero": "Desconocido", "edad": 0})
display(df_flags)
</pre>

<pre>
# 2. Estandarización: convertir a minúsculas y normalizar nombres de ciudad
df_estandar = df_limpio.withColumn("nombre", lower(col("nombre"))) \
                       .withColumn("ciudad", regexp_replace(lower(col("ciudad")), "madrid", "MADRID"))
display(df_flags)
</pre>

<pre>
# 3. Reglas de negocio: crear flags
df_flags = df_estandar.withColumn("es_madrid", when(col("ciudad") == "MADRID", 1).otherwise(0)) \
                      .withColumn("mayor_edad", when(col("edad") >= 18, 1).otherwise(0)) \
                      .withColumn("registro_reciente", when(col("fecha_registro") >= "2026-01-01", 1).otherwise(0))

display(df_flags)
</pre>

%md
## 4️⃣ ¿Por qué Delta Lake?

Delta Lake es una capa de almacenamiento open source que se integra con Apache Spark y añade capacidades como:

- Transacciones **ACID**:
    - **Atomicity**: todo o nada
    - **Consistency**: el esquema se respeta
    - **Isolation**: escrituras concurrentes
    - **Durability**: los datos persisten
- **Schema enforcement**
- **Time Travel**: Delta guarda un **log de transacciones**. Permite:
    - Auditar cambios
    - Recuperar datos
    - Comparar versiones
- Optimización de datos

👉 Sin Delta Lake, un Data Lake es solo almacenamiento

##### ¿Por qué es tan valioso el formato de Delta Table?
Guardar tablas en formato Delta permite aprovechar las ventajas de transacciones ACID, manejo de versiones, y optimización de consultas. Es ideal para entornos donde los datos cambian frecuentemente y se requiere trazabilidad.

Una Delta Table permite realizar operaciones ACID, mantener el histórico con time travel, gestionar versiones, optimizar el almacenamiento, y escalar en entornos de producción.

### Guardar una tabla
Guardar una tabla en Databricks puede hacerse de dos formas principales:


%md
| Método | Descripción | ✔️ Ventajas | Uso Recomendado |
|--------|-------------|----------|-----------------|
| **Por path** | Guarda los datos en una ruta específica del sistema de archivos (ej: DBFS) usando formato Delta. | - Flexible y directo<br>- Fácil de mover entre entornos<br>- Integración con sistemas externos | Automatización, migraciones, acceso directo por ruta |
| **Por catálogo del metastore** | Registra la tabla en el catálogo de Databricks para consultas SQL y control de acceso. | - Ideal para colaboración multiusuario<br>- Control de permisos y versiones<br> - Auditoría y seguridad avanzada<br>- Fácil acceso mediante SQL | Entornos colaborativos, equipos de análisis, integración con BI |

**Nota:** Ambas opciones aprovechan las ventajas del formato Delta: transacciones ACID, versionado y optimización de consultas.




Por path
<pre>
df_csv.write.format("delta").mode("overwrite").save(...)
</pre>

Por catalogo
<pre>
df_csv.write.format("delta").mode("overwrite").saveAsTable("...")
</pre>

### Leer una tabla

%md
1. Lectura de una tabla Delta desde Spark: También puedes leer una tabla Delta directamente desde Spark usando el API de DataFrame:

<pre>
df_delta = spark.read.format("delta").load(base_path+"delta")
df_delta.show()
</pre>

Esto es útil cuando necesitas manipular los datos con Python, realizar transformaciones complejas, aplicar lógica de negocio o integrarlo en pipelines de procesamiento.

2. Consulta SQL sobre una tabla Delta: Puedes consultar una tabla Delta registrada en el catálogo usando SQL estándar. Por ejemplo:

<pre>
SELECT * FROM ceste.productos;
</pre>

Esto te permite aprovechar toda la potencia del lenguaje SQL para filtrar, agrupar, unir y analizar los datos almacenados en formato Delta. Es especialmente útil para usuarios que prefieren trabajar con SQL o para integraciones con herramientas de BI.

%md
| Método                |       ✔️ Ventajas        | Diferencias| Uso recomendado|
|-----------------------|--------------------|------------|----------------|
| **Spark DataFrame API** | - Procesamiento avanzado<br>- Integración con ML y ETL<br>- Automatización y pipelines<br>- Flexibilidad en transformaciones                         | Permite lógica compleja y manipulación programática de datos.  | Procesos automáticos, machine learning, ETL, integración con Python/Scala.|
| **SQL**               | - Fácil de usar y compartir<br>- Ideal para análisis exploratorio<br>- Integración con dashboards y BI<br>- Colaboración multiusuario         | Sintaxis declarativa, acceso directo desde notebooks y herramientas BI.| Análisis, reporting, dashboards, colaboración entre equipos.     |

Ambos métodos aprovechan las ventajas de Delta Lake: transacciones ACID, versionado, rendimiento y escalabilidad.


### Modificar una tabla
Cuando trabajamos con tablas Delta, una de las grandes ventajas es la posibilidad de realizar operaciones transaccionales complejas de forma eficiente y segura como:
- Updates
- Deletes
- Merges

<pre>
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, base_path+"delta")
</pre>

#### Update
%md
Permite modificar el valor de una o varias columnas en las filas que cumplen una condición específica. Por ejemplo, se puede actualizar el nombre de un producto o corregir un valor erróneo en una tabla sin tener que reescribir todo el dataset. La sintaxis es similar a la de SQL, pero se realiza sobre la API de DeltaTable en Spark.

<pre>
# UPDATE
delta_table.update(
    condition="id = 1",
    set={"producto": "'Ordenador'"}
)
</pre>

#### Delete
Permite eliminar filas que cumplen una condición determinada. Es útil para depurar datos, eliminar registros obsoletos o cumplir con requisitos legales de borrado. Al igual que el update, el delete es transaccional y garantiza la integridad de la tabla.

<pre>
delta_table.delete("precio > 150")
</pre>
%md
Ambas operaciones aprovechan las transacciones ACID de Delta Lake, lo que significa que los cambios son atómicos, consistentes, aislados y duraderos. Esto evita problemas de concurrencia y asegura que los datos siempre estén en un estado válido, incluso en entornos multiusuario o de procesamiento distribuido.


#### Merge
En este bloque de código se muestra cómo realizar un "merge" (también conocido como upsert) sobre una tabla Delta:

1. Se crea un DataFrame con nuevos datos o datos actualizados.
2. Luego, se utiliza el método `merge` de la API de Delta Lake para comparar los datos existentes en la tabla (target) con los nuevos datos (source) usando una condición de emparejamiento (en este caso, el campo id).
3. Si el `id` ya existe en la tabla, se actualizan todos los campos de ese registro (`whenMatchedUpdateAll`).
4. Si el `id` no existe, se inserta el nuevo registro (`whenNotMatchedInsertAll`).

<pre>
# Nuevos datos a insertar/actualizar
columns = ["id", "fecha", "producto", "cantidad", "precio"]

nuevos_datos = [(3, "2025-05-24", "Monitor", 1, 179.99), (4, "2025-05-24", "Impresora", 2, 89.99)]
df_updates = spark.createDataFrame(nuevos_datos, columns)


delta_table.alias("target").merge(
    df_updates.alias("source"),
    "target.id = source.id") \
  .whenMatchedUpdateAll() \
  .whenNotMatchedInsertAll() \
  .execute()
  </pre>

Este tipo de operación es fundamental en escenarios de integración incremental de datos, donde periódicamente llegan nuevos registros o actualizaciones y queremos mantener la tabla Delta siempre actualizada y sin duplicados.

Ventajas de usar merge en Delta Lake:

- Permite mantener la integridad y consistencia de los datos.
- Facilita la implementación de pipelines de datos incrementales.
- Aprovecha las transacciones ACID de Delta Lake, evitando problemas de concurrencia o corrupción de datos.
- Es mucho más eficiente y sencillo que realizar operaciones manuales de actualización e inserción por separado.


### Time Travel

El Time Travel en Delta Lake es una funcionalidad que permite consultar versiones anteriores de una tabla Delta. Cada vez que se realiza una operación de escritura (insert, update, delete, merge), Delta Lake crea una nueva versión de la tabla, manteniendo el historial de cambios.

**¿Para qué sirve el Time Travel?**
- Recuperar datos borrados o modificados accidentalmente.
- Auditar cambios y analizar cómo han evolucionado los datos a lo largo del tiempo.
- Comparar el estado de la tabla en diferentes momentos.
- Reproducir experimentos o análisis sobre datos históricos.

**¿Cómo se usa?**
Puedes acceder a una versión anterior de la tabla especificando el número de versión (`versionAsOf`) o una marca de tiempo (`timestampAsOf`) al leer los datos:

**Ventajas:**
- No necesitas mantener copias manuales de los datos para auditoría o recuperación.
- Todas las operaciones de Time Travel son transaccionales y consistentes.
- Facilita la trazabilidad y el cumplimiento normativo en entornos empresariales.

Copia las siguientes celdas en el Notebook y mira lo que pasa

Ver historial
<pre>
display(delta_table.history())
</pre>

Leer versión anterior
<pre>
df_old = spark.read.format("delta").option("versionAsOf", 0).load(base_path+"delta")
display(df_old)
</pre>

Leer la tabla tal como estaba en una fecha concreta
<pre>
df_moment = spark.read.format("delta").option("timestampAsOf", "2025-05-27 10:00:00").load(base_path+"delta")
</pre>


### Optimización de Tablas
%md
La optimización de tablas en Delta Lake es clave para mejorar el rendimiento de las consultas y reducir el coste de almacenamiento en entornos de Big Data. Existen dos técnicas principales:

- **Compactación (OPTIMIZE):** Consiste en reducir el número de archivos pequeños que se generan tras múltiples escrituras o actualizaciones. Al compactar, se agrupan estos archivos en otros más grandes, lo que acelera las lecturas y reduce la sobrecarga de gestión de archivos en el sistema distribuido.

- **Z-Ordering:** Es una técnica de ordenación física de los datos en disco basada en una o varias columnas clave. Al aplicar Z-Ordering, los datos se almacenan de forma que las filas con valores similares en las columnas seleccionadas queden físicamente próximas. Esto mejora notablemente el rendimiento de las consultas filtradas por esas columnas, ya que minimiza la cantidad de datos que Spark necesita leer.

**Ventajas de la optimización:**
- Consultas más rápidas y eficientes, especialmente en grandes volúmenes de datos.
- Menor latencia en dashboards y análisis interactivos.
- Reducción de costes de almacenamiento y procesamiento.
- Mejor aprovechamiento de los recursos del cluster.

**Cuándo optimizar:**
- Tras cargas masivas de datos o procesos ETL frecuentes.
- Cuando se detecta degradación en el rendimiento de las consultas.
- Antes de ejecutar análisis críticos o dashboards de negocio.

En resumen, la optimización periódica de las tablas Delta es una buena práctica para mantener el entorno ágil, eficiente y escalable.

Optimizar tabla
<pre>
spark.sql("OPTIMIZE ceste.productos")
</pre>

Ordenar físicamente por "id"
<pre>
spark.sql("OPTIMIZE ceste.productos ZORDER BY id")
</pre>

%md
## 5️⃣ Transacciones ACID (muy importante)

- **Atomicity**: todo o nada
- **Consistency**: el esquema se respeta
- **Isolation**: escrituras concurrentes
- **Durability**: los datos persisten

👉 Clave para entornos productivos


%md
## 6️⃣ Time Travel

Delta guarda un **log de transacciones**.

Permite:
- Auditar cambios
- Recuperar datos
- Comparar versiones

👉 Muy usado para debugging y errores humanos


---

%md
## 7️⃣ Modelado y pipelines

Arquitectura típica:

### Bronze
- Datos crudos

### Silver
- Datos limpios y transformados

### Gold
- Datos agregados y listos para negocio

👉 Cada capa suele ser un notebook


%md
## 8️⃣ Jobs y Workflows

### Job
- Ejecuta un notebook
- Programado o manual

### Workflow
- Encadena múltiples Jobs
- Controla dependencias
- Maneja errores

👉 Es lo que se usa en producción


%md
## ✅ Mensaje final

Un buen pipeline:
- Es **automático**
- Es **reproducible**
- Falla de forma controlada
- Está preparado para crecer
